# Árvores de regressão - exercícios 01

Nesta bateria de exercícios, vamos botar em prática o que aprendemos com a base de dados imobiliários de Boston, que pode ser baixada do módulo ```datasets``` do scikitlearn. Essa base de dados possui uma linha por cidade, o objetivo é prever, ou fornecer uma estimativa de ```median_house_value```, ou o valor mediano de casas ocupadas pelo dono, dadas as demais variáveis usadas como explicativas.

A descrição das variáveis está abaixo:

| Variavel | Descrição | Tipo |
|-|-|-|
|longitude| Longitude | Contínua |
|latitude| Latitude | Contínua |
|housingmedianage| Idade mediana das casas nesta área | Contínua |
|total_rooms| Número médio de ambientes por habitação nesta área | Contínua |
|total_bedrooms| Número médio de ambientes por habitação nesta área | Contínua |
|population| Número de pessoas nesta área | Contínua |
|households| Número de grupos familiares nesta área | Contínua |
|median_income| Renda mediana do grupo familiar | Contínua |
|median_house_value| (variável resposta) Valor mediano das casas | Contínua |
|ocean_proximity| Proximidade com o oceâno | Categórica |

In [97]:
import pandas as pd

import seaborn as sns

from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split

### 1. Certifique-se de que esta base está no formato adequado para o scikitlearn.
ok, essa tarefa é tão fácil que você vai até desconfiar. Mas é preciso ter confiança sobre os dados que se usa ;)

In [150]:
df = pd.read_csv('housing.csv', delimiter = ',')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [154]:
df.shape

(20640, 10)

In [133]:
X = df.drop(columns = ['median_house_value']).copy()
y = df[['median_house_value']].copy()

In [162]:
X.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY


In [135]:
X.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity'],
      dtype='object')

In [137]:
y.columns

Index(['median_house_value'], dtype='object')

In [139]:
y.head()

,median_house_value
0,452600.0
1,358500.0
2,352100.0
3,341300.0
4,342200.0


In [181]:
metadados_X = pd.DataFrame({'dtypes': X.dtypes})
metadados_X['missing'] = X.isnull().sum()
metadados_X['perc_missing'] = round((metadados_X['missing']/df.shape[0])*100)
metadados_X['valores_unicos'] = X.nunique()

metadados_X

,dtypes,missing,perc_missing,valores_unicos
longitude,float64,0,0.0,844
latitude,float64,0,0.0,862
housing_median_age,float64,0,0.0,52
total_rooms,float64,0,0.0,5926
total_bedrooms,float64,207,1.0,1923
population,float64,0,0.0,3888
households,float64,0,0.0,1815
median_income,float64,0,0.0,12928
ocean_proximity,object,0,0.0,5


In [183]:
metadados_y = pd.DataFrame({'dtypes': y.dtypes})
metadados_y['missing'] = y.isnull().sum()
metadados_y['perc_missing'] = round((metadados_y['missing']/df.shape[0])*100)
metadados_y['valores_unicos'] = y.nunique()
metadados_y

,dtypes,missing,perc_missing,valores_unicos
median_house_value,float64,0,0.0,3842


In [193]:
X.dropna(axis = 0, inplace = True)
X

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,INLAND


### 2.  Visualize a matriz de correlação, e veja se aparentemente você encontra alguma variável com potencial preditivo interessante.

### 3. Separe os dados em validação e teste

### 4. Treine duas árvores, uma com profundidade máxima = 8, outra com profundidade máxima = 2.

### 5. Calcule o MSE de cada uma das árvores do item anterior para a base de treinamento e para a base de testes.

### 6. Com base nos resultados do item anterior, qual árvore te parece mais adequada?

### 7. Faça uma visualização gráfica dessa árvore. Vamos discutir importância de variável mais adiante, mas veja a sua árvore, e pense um pouco: qual variável te parece mais "importante" na árvore?